# sqlite

> sqlite storage for rdflib knowledge graphs

In [ ]:
#| default_exp sqlite

# RDFLib Parquet Storage

This module provides a fast, efficient way to store and retrieve RDF graphs using Parquet files. It wraps RDFLib's graph functionality with optimized Parquet serialization methods.

## Key Features

- Save RDF graphs to compressed Parquet files
- Load graphs from Parquet with optimized performance
- Preserve all RDF semantics (URIs, blank nodes, literals with datatypes)
- Memory-efficient batch processing for large graphs
- Fluent API for method chaining
- Simple integration with existing RDFLib code

## Use Cases

- Store large knowledge graphs efficiently
- Improve load/save times for RDF data
- Integrate semantic web data with data science pipelines
- Reduce storage requirements for RDF datasets
- Enable faster querying through columnar storage benefits

This module bridges the gap between semantic web technologies and modern data engineering practices, allowing RDF data to benefit from Parquet's columnar storage format advantages including compression, schema enforcement, and performance.

This is inspired by KGLabs implimentation [Performance analysis of serialization methods¶](https://derwen.ai/docs/kgl/ex2_0/#performance-analysis-of-serialization-methods) that showed parquet to be a reasonable triple store for local graphs. This also provides some convenience functions for constructing KGs.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import sqlite3
import rdflib
from rdflib.store import Store, NO_STORE, VALID_STORE
from rdflib import URIRef, Literal, BNode, Graph
from fastcore.all import *
from fastkg.core import KnowledgeGraph

In [ ]:
#| export
class SQLiteStore(Store):
    """
    Simple SQLite-based triple store for RDFLib
    """
    
    def __init__(self, configuration=None):
        super(SQLiteStore, self).__init__()
        self.configuration = configuration
        self.conn = None
        self.cursor = None
    
    def open(self, configuration, create=False):
        """
        Open the SQLite database. If create is True, create the 
        database if it doesn't exist.
        """
        if not configuration:
            configuration = self.configuration
        
        if not os.path.exists(configuration) and not create:
            return NO_STORE
        
        self.conn = sqlite3.connect(configuration)
        self.cursor = self.conn.cursor()
        
        # Create tables if needed
        if create:
            self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS triples (
                subject TEXT NOT NULL,
                predicate TEXT NOT NULL,
                object TEXT NOT NULL,
                UNIQUE(subject, predicate, object)
            )
            """)
            
            self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_s ON triples (subject)")
            self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_p ON triples (predicate)")
            self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_o ON triples (object)")
            self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_sp ON triples (subject, predicate)")
            self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_so ON triples (subject, object)")
            self.cursor.execute("CREATE INDEX IF NOT EXISTS idx_po ON triples (predicate, object)")
            
            self.conn.commit()
        
        return VALID_STORE
    
    def close(self, commit_pending_transaction=False):
        """Close the database connection"""
        if commit_pending_transaction:
            self.conn.commit()
        self.conn.close()
        self.conn = None
        self.cursor = None
    
    def _term_to_string(self, term):
        """Convert an RDFLib term to a string for storage"""
        if isinstance(term, URIRef):
            return f"U:{term}"
        elif isinstance(term, BNode):
            return f"B:{term}"
        elif isinstance(term, Literal):
            if term.language:
                return f"L:{term}@{term.language}"
            elif term.datatype:
                return f"L:{term}^^{term.datatype}"
            else:
                return f"L:{term}"
        return str(term)
    
    def _string_to_term(self, string):
        """Convert a stored string back to an RDFLib term"""
        if string.startswith("U:"):
            return URIRef(string[2:])
        elif string.startswith("B:"):
            return BNode(string[2:])
        elif string.startswith("L:"):
            # Handle language tags and datatypes
            if "^^" in string:
                value, datatype = string[2:].rsplit("^^", 1)
                return Literal(value, datatype=URIRef(datatype))
            elif "@" in string:
                value, lang = string[2:].rsplit("@", 1)
                return Literal(value, lang=lang)
            else:
                return Literal(string[2:])
        return string
        
    def add(self, triple, context=None, quoted=False):
        """Add a triple to the store"""
        # Check if connection is open
        if self.cursor is None:
            raise RuntimeError("Database connection is closed")
        
        # We'll ignore the context parameter but raise an error for quoted statements
        if quoted:
            raise NotImplementedError("Quoted statements not supported")
        
        s, p, o = triple
        s_str = self._term_to_string(s)
        p_str = self._term_to_string(p)
        o_str = self._term_to_string(o)
        
        self.cursor.execute(
            "INSERT OR IGNORE INTO triples (subject, predicate, object) VALUES (?, ?, ?)",
            (s_str, p_str, o_str)
        )
        self.conn.commit()

        

    def remove(self, triple_pattern, context=None):
        """Remove triples matching the pattern"""
        # We'll ignore the context parameter
        s, p, o = triple_pattern
        params = []
        where_clauses = []
        
        if s is not None:
            where_clauses.append("subject = ?")
            params.append(self._term_to_string(s))
        
        if p is not None:
            where_clauses.append("predicate = ?")
            params.append(self._term_to_string(p))
        
        if o is not None:
            where_clauses.append("object = ?")
            params.append(self._term_to_string(o))
        
        if where_clauses:
            query = f"DELETE FROM triples WHERE {' AND '.join(where_clauses)}"
            self.cursor.execute(query, params)
        else:
            # Remove all triples if no pattern specified
            self.cursor.execute("DELETE FROM triples")
        
        self.conn.commit()

    def triples(self, triple_pattern, context=None):
        """
        Generator over triples matching the pattern.
        Returns (triple, context) tuples.
        """
        # Check if connection is open
        if self.cursor is None:
            return  # Empty generator if connection is closed
        
        # We'll ignore the context parameter
        s, p, o = triple_pattern
        params = []
        where_clauses = []
        
        if s is not None:
            where_clauses.append("subject = ?")
            params.append(self._term_to_string(s))
        
        if p is not None:
            where_clauses.append("predicate = ?")
            params.append(self._term_to_string(p))
        
        if o is not None:
            where_clauses.append("object = ?")
            params.append(self._term_to_string(o))
        
        if where_clauses:
            query = f"SELECT subject, predicate, object FROM triples WHERE {' AND '.join(where_clauses)}"
        else:
            query = "SELECT subject, predicate, object FROM triples"
        
        self.cursor.execute(query, params)
        
        for s_str, p_str, o_str in self.cursor.fetchall():
            s = self._string_to_term(s_str)
            p = self._string_to_term(p_str)
            o = self._string_to_term(o_str)
            yield (s, p, o), None


    def __len__(self, context=None):
        """Return the number of triples in the store"""
        # We'll ignore the context parameter
        if self.cursor is None:
            return 0  # Return 0 if the connection is closed
        
        self.cursor.execute("SELECT COUNT(*) FROM triples")
        return self.cursor.fetchone()[0]


In [ ]:
#| export
# Register our SQLite store with RDFLib
rdflib.plugin.register(
    'SQLite', rdflib.store.Store,
    'fastkg.sqlite', 'SQLiteStore'
)

In [ ]:
#| export
@patch
def connect_sqlite(self:KnowledgeGraph, db_path, create=True):
    """Connect to a SQLite database file"""
    # Create a new graph with our SQLiteStore directly
    store = SQLiteStore()
    self.g = Graph(store)
    
    # Open the database
    self.g.open(db_path, create=create)
    
    return self


In [ ]:
#| export
@patch
def close(self:KnowledgeGraph):
    """Close the database connection if using a persistent store"""
    if hasattr(self.g.store, 'close'):
        self.g.close()
    return self

In [ ]:
#| hide
# Helper function to clean up test databases
def remove_test_db(path):
    if os.path.exists(path):
        os.remove(path)


In [ ]:
# Step 1: Test basic functionality
db_path = "test_sqlite_store.db"
remove_test_db(db_path)  # Clean up any existing test database

# Create a new store
store = SQLiteStore()

# Open the store
result = store.open(db_path, create=True)
print(f"Open result: {result}")

# Add a triple
test_triple = (
    URIRef("http://example.org/subject"),
    URIRef("http://example.org/predicate"),
    Literal("test object")
)
store.add(test_triple)

# Check if the triple was added
print(f"Store length: {len(store)}")

# Query for the triple
results = list(store.triples((None, None, None)))
print(f"All triples: {results}")

# Close the store
store.close()


Open result: 1
Store length: 1
All triples: [((rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/predicate'), rdflib.term.Literal('test object')), None)]


In [ ]:
# Step 2: Test query patterns
store = SQLiteStore()
store.open(db_path)

# Add more triples for testing queries
store.add((
    URIRef("http://example.org/subject"),
    URIRef("http://example.org/another-predicate"),
    Literal("another object")
))

store.add((
    URIRef("http://example.org/another-subject"),
    URIRef("http://example.org/predicate"),
    Literal("third object")
))

# Query with subject pattern
print("Query with subject pattern:")
results = list(store.triples((URIRef("http://example.org/subject"), None, None)))
for triple, ctx in results:
    print(f"  {triple}")

# Query with predicate pattern
print("\nQuery with predicate pattern:")
results = list(store.triples((None, URIRef("http://example.org/predicate"), None)))
for triple, ctx in results:
    print(f"  {triple}")

# Query with object pattern
print("\nQuery with object pattern:")
results = list(store.triples((None, None, Literal("test object"))))
for triple, ctx in results:
    print(f"  {triple}")

# Query with subject-predicate pattern
print("\nQuery with subject-predicate pattern:")
results = list(store.triples((
    URIRef("http://example.org/subject"),
    URIRef("http://example.org/predicate"),
    None
)))
for triple, ctx in results:
    print(f"  {triple}")

store.close()


Query with subject pattern:
  (rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/another-predicate'), rdflib.term.Literal('another object'))
  (rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/predicate'), rdflib.term.Literal('test object'))

Query with predicate pattern:
  (rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/predicate'), rdflib.term.Literal('test object'))
  (rdflib.term.URIRef('http://example.org/another-subject'), rdflib.term.URIRef('http://example.org/predicate'), rdflib.term.Literal('third object'))

Query with object pattern:
  (rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/predicate'), rdflib.term.Literal('test object'))

Query with subject-predicate pattern:
  (rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/predicate'), rdflib.term.Literal('test object'))


In [ ]:
# Step 3: Test different RDF term types
store = SQLiteStore()
store.open(db_path)

# Add triples with different term types
# Blank node
bnode = BNode()
store.add((bnode, URIRef("http://example.org/type"), Literal("blank node")))

# Literal with language tag
store.add((
    URIRef("http://example.org/subject"),
    URIRef("http://example.org/label"),
    Literal("hello", lang="en")
))

# Literal with datatype
store.add((
    URIRef("http://example.org/subject"),
    URIRef("http://example.org/age"),
    Literal("42", datatype=URIRef("http://www.w3.org/2001/XMLSchema#integer"))
))

# Query for specific term types
print("Blank node triples:")
for triple, ctx in store.triples((None, URIRef("http://example.org/type"), None)):
    print(f"  {triple}")
    print(f"  Subject type: {type(triple[0])}")

print("\nLanguage-tagged literal:")
for triple, ctx in store.triples((None, URIRef("http://example.org/label"), None)):
    print(f"  {triple}")
    print(f"  Object language: {triple[2].language}")

print("\nDatatyped literal:")
for triple, ctx in store.triples((None, URIRef("http://example.org/age"), None)):
    print(f"  {triple}")
    print(f"  Object datatype: {triple[2].datatype}")

store.close()


Blank node triples:
  (rdflib.term.BNode('Nc109a8aa596b4894b86367280c0726cc'), rdflib.term.URIRef('http://example.org/type'), rdflib.term.Literal('blank node'))
  Subject type: <class 'rdflib.term.BNode'>

Language-tagged literal:
  (rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/label'), rdflib.term.Literal('hello', lang='en'))
  Object language: en

Datatyped literal:
  (rdflib.term.URIRef('http://example.org/subject'), rdflib.term.URIRef('http://example.org/age'), rdflib.term.Literal('42', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')))
  Object datatype: http://www.w3.org/2001/XMLSchema#integer


In [ ]:
# Step 4: Test removal
store = SQLiteStore()
store.open(db_path)

# Count triples before removal
print(f"Triples before removal: {len(store)}")

# Remove a specific triple
store.remove((
    URIRef("http://example.org/subject"),
    URIRef("http://example.org/predicate"),
    Literal("test object")
))

print(f"Triples after specific removal: {len(store)}")

# Remove triples matching a pattern
store.remove((URIRef("http://example.org/subject"), None, None))

print(f"Triples after pattern removal: {len(store)}")

# Check remaining triples
print("Remaining triples:")
for triple, ctx in store.triples((None, None, None)):
    print(f"  {triple}")

store.close()


Triples before removal: 6
Triples after specific removal: 5
Triples after pattern removal: 2
Remaining triples:
  (rdflib.term.URIRef('http://example.org/another-subject'), rdflib.term.URIRef('http://example.org/predicate'), rdflib.term.Literal('third object'))
  (rdflib.term.BNode('Nc109a8aa596b4894b86367280c0726cc'), rdflib.term.URIRef('http://example.org/type'), rdflib.term.Literal('blank node'))


In [ ]:
# Test with KnowledgeGraph
kg_db_path = "test_kg_sqlite.db"
remove_test_db(kg_db_path)  # Clean up any existing test database

# Create a new graph
kg = KnowledgeGraph()
kg.connect_sqlite(kg_db_path)

# Add some data
ex = rdflib.Namespace("http://example.org/")
kg.bind_ns("ex", ex)
kg.add((ex.John, rdflib.RDF.type, ex.Person))
kg.add((ex.John, ex.name, rdflib.Literal("John Doe")))
kg.add((ex.John, ex.age, rdflib.Literal(30)))

print(f"Added {len(kg)} triples to SQLite database")

# Run a SPARQL query
q = """
SELECT ?name WHERE {
  ?person a <http://example.org/Person> .
  ?person <http://example.org/name> ?name .
}
"""
results = list(kg.query(q))
print(f"Query result: {results[0][0] if results else 'No results'}")

# Close the connection
kg.close()

# Connect to the same DB with a new graph
kg2 = KnowledgeGraph()
kg2.connect_sqlite(kg_db_path, create=False)
print(f"Loaded graph has {len(kg2)} triples")

# Run the same query
results = list(kg2.query(q))
print(f"Query result after reload: {results[0][0] if results else 'No results'}")

kg2.close()

Added 3 triples to SQLite database
Query result: John Doe
Loaded graph has 3 triples
Query result after reload: John Doe


KnowledgeGraph(triples=0)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()